In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

In [2]:
from architectures.lstm import SimpleLSTM
from architectures.gpt import GPTDecoder

vocab_size = 50_000   # number of tokens
embed_dim = 384     # embedding dimension
hidden_dim = 384     # LSTM hidden size
num_layers = 2

lstm = SimpleLSTM(vocab_size, embed_dim, hidden_dim, num_layers)
param_count = sum(p.numel() for p in lstm.parameters() if p.requires_grad)
print(f"LSTM has {param_count} trainable params")

LSTM has 21615440 trainable params


In [3]:
vocab_size = 50_000
embed_dim = 256
num_heads = 8
ff_hidden_dim = 2048
num_layers = 6
context_length = 128
dropout = 0.1

gpt = GPTDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_hidden_dim=ff_hidden_dim,
    num_layers=num_layers,
    context_length=context_length,
    dropout=dropout
)

param_count = sum(p.numel() for p in gpt.parameters())
print(f"GPT has {param_count} trainable params")

GPT has 20690944 trainable params


In [10]:
import torch

def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [11]:
import time
from functools import wraps

def measure_inference_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        elapsed = end - start
        print(f"[{func.__name__}] Inference time: {elapsed:.4f} seconds")
        return result
    return wrapper

In [42]:
from torch.functional import F

@measure_inference_time
@torch.no_grad()
def generate_text_lstm(model, tokenizer, prompt, max_new_tokens=20, temperature: float = 1.0, device=None):
    if device is None:
        device = choose_device()
    
    model.to(device)
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    generated_tokens = tokens.copy()

    with torch.no_grad():
        hidden = None
        for _ in range(max_new_tokens):
            out, hidden = model(input_ids, hidden)

            logits = out[0, -1, :]
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)

            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
            input_ids = torch.tensor([[next_token_id]], dtype=torch.long).to(device)

            generated_tokens.append(next_token_id.item())
    
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

@measure_inference_time
@torch.no_grad()
def generate_text_gpt(model, tokenizer, prompt, max_new_tokens=20,  temperature=1.0, device=None):
    if device is None:
        device = choose_device()

    model.to(device)
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)

    for _ in range(max_new_tokens):
        logits = model(input_ids)

        logits = logits / temperature
        logits = logits[0, -1, :]
        
        probs = F.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

    generated_tokens = input_ids[0].tolist()
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

In [37]:
lstm.load_state_dict(torch.load("saved_models/lstm_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [38]:
gpt.load_state_dict(torch.load("saved_models/gpt_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [39]:
prompts = [
    "Czasem jedno słowo potrafi zmienić cały dzień.",
    "Wczoraj ktoś zostawił mi kartkę na ławce, bez podpisu.",
    "No dobra, ale kto w ogóle uznał, że to ma sens?",
    "To miało być tylko na chwilę, a wyszło jak zawsze.",
    "Nie wiem, czy to przez pogodę, czy przez ludzi, ale dziś wszystko wydaje się dziwnie ciche.",
    "„Nie klikaj tam” — powiedział, zanim ekran zgasł.",
    "W sumie nie planowałem o tym mówić, ale skoro już tu jesteś…",
    "Dwa dni bez snu i nagle wszystko zaczyna się układać. Ironia, co?",
    "Kiedy byłem mały, myślałem, że dorośli wszystko wiedzą.",
    "Czasami po prostu trzeba usiąść, włączyć coś spokojnego i udawać, że świat się nie pali."
]

In [43]:
import textwrap

for prompt in prompts:
    text = generate_text_gpt(gpt, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
    print(textwrap.fill(text, width=80), "\n")

[generate_text_gpt] Inference time: 1.2900 seconds
Czasem jedno słowo potrafi zmienić cały dzień . Oficjalna strona na język
niemieckim trafiło na swoje perskie określenie , ale były używane w języku
rosyjskim . Jeśli ktokolwiek dowodził , byli powracający do Port Royal : ff Nowy
Jork ” ( w Wenezueli wygrana 1 : 2 ) . Masy ten dotyczy formalnie zaradłego
narodu w Polsce . Piraci Urugwaj brali również udział w wielu rozgrywkach , m .
in . w roku 1976 po raz pierwszy w historii jako autor nu , który zajmuje mniej
swoich potencjalnych kibiców . Na arenie międzynarodowej flaga argentyńska
wywoziona 

[generate_text_gpt] Inference time: 1.3211 seconds
Wczoraj ktoś zostawił mi kartkę na ławce , bez podpisu . W dawnej fy komiks "
pioruny u wikaczy , tak więc Jakub został obaj zagwarantowany trzem jednostkom
niemieckiej policji , która korzystała z niej wielu oficerów . EifflenHugo
Essenberg ( ur . 3 kwietnia 1932 w Lünestann , zm . 3 grudnia 2019 w Moskwie ) –
belgijski reżyser filmowy , teat

In [47]:
for prompt in prompts:
    text = generate_text_lstm(lstm, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
    print(textwrap.fill(text, width=80), "\n")

[generate_text_lstm] Inference time: 0.2261 seconds
Czasem jedno słowo potrafi zmienić cały dzień . Trybuny ekonomiczne starają się
unikać , aby wymiar sprawiedliwości , trujący się zdozna do wykonywania ataków
na cele wydawnicze , np . przy jednoczesnym zachowaniu wykształcenia
stomatologii , nauczyciele , którzy prześladowali jakiekolwiek włoskie i
kulturowe fyllekspitów oraz chcąc wrócić do życia , pierwszym i w efekcie
doradzał antyrządowi tiony ciesiewczego przy operacji negocjacji , którą
działalność jest ostatecznie tyczszy , a efekt nowego neutronu . W dobie
stalinizmu . Tymczasem Metałocito rozum . W Polsce 

[generate_text_lstm] Inference time: 0.2104 seconds
Wczoraj ktoś zostawił mi kartkę na ławce , bez podpisu . Latem 2006 przy
szpitalu tego kursu w walce żyją o jeden rok , w " Włostmuntowni jach " pisano
na własnym boisku pierwszą dyszę . W grudniu 2019 wstąpiła do czołowych krajów
socjalistycznych z Mściwoj I . W 1997 działacze Barbary Sośnicko wniosek Jaklubu
Anatyjskie